In [1]:
import pandas as pd
import numpy as np

In [2]:
k = 0.8
Iplan = 100
ventas = [10,15,20,5,10,25,10,15,20,5] * 3
demora = [5,3,4,6,2,3,1,5,4,5] * 3

def calc_pedido(k, V, I, y, X):
    #if X[-1] > 60:
     #   return 0
    if len(X) == 3:
        weighted_avg = 0.1 * X[0] + 0.3 * X[1] + 0.5 * X[2] 
        if weighted_avg > 25:
            return 0   
        else:
            return V + k * (I - y)
    else:
        return V + k * (I - y)

def reposicion_ventas(Iplan, Y, X):
    reposicion = 0
    if Y < 80:
            reposicion = Iplan - Y
    elif (Y > 105):
        reposicion = 0
    else:
        reposicion = np.random.normal(loc=np.abs(Iplan-Y),scale=np.abs(Iplan-Y)/6)
    if reposicion > X:
        return X
    return reposicion
def costo_mantenimiento(X):
    return X*10
def delivery(spreadsheet, index):
    indexes = (np.arange(0,index).tolist()[::-1])
    count = 1
    acum = 0
    for idx in indexes:
        if(spreadsheet['Demora'].values[idx] == count):
            acum += spreadsheet['Pedido'].values[idx]
        count += 1
    return acum
keys = ['Tiempo','Ventas','Demora','Llegada','Pedido','Reposicion','Y','X','CP','CM','CT']

In [3]:
ss_dict = {
    'Tiempo':[],
    'Ventas':[],
    'Demora':[],
    'Llegada':[],
    'Pedido':[],
    'Reposicion':[],
    'Y':[],
    'X':[],
    'CP':[],
    'CM':[],
    'CT':[],    
}

In [4]:
spreadsheet = pd.DataFrame(ss_dict)
spreadsheet

,Tiempo,Ventas,Demora,Llegada,Pedido,Reposicion,Y,X,CP,CM,CT


In [5]:
def step(spreadsheet, index):
    temp = []
    #tiempo
    temp.append(index)
    if index == 0:
        #V
        temp.append(0)
        #D
        temp.append(0)
        #llegada
        temp.append(0)
        #pedido
        temp.append(0)
        #Reposicion
        temp.append(0)
        #Y
        temp.append(Iplan)
        #X
        X = np.mean(ventas) * np.mean(demora)
        temp.append(X)
        #CP
        temp.append(0)
        #CM
        temp.append(costo_mantenimiento(X))
        #CT
        temp.append(temp[-1] + temp[-2] + (X + Iplan) * 100)
    else:
        #V
        temp.append(ventas[index-1])
        #D
        temp.append(demora[index-1])
        #llegada
        llega_stock = delivery(spreadsheet, index) 
        temp.append(llega_stock)
        #Y
        Y = spreadsheet['Y'].values[-1] - ventas[index-1]
        #pedido
        X = spreadsheet['X'].values[-1] + llega_stock

        P = calc_pedido(k, ventas[index], Iplan, Y, spreadsheet['X'].values[-3::])
        temp.append(P)
        #Reposicion

        R = reposicion_ventas(Iplan, Y, X)
        X = spreadsheet['X'].values[-1] - R + llega_stock
        temp.append(R)
        #Y asignacion
        temp.append(Y+R)
        #X
        temp.append(X)
        #CP
        if P !=0:
            CP = 1000 + 100*P
            temp.append(CP)
        else:
            temp.append(0)
        #CM
        temp.append(costo_mantenimiento(X))
        #CT
        temp.append(temp[-1] + temp[-2])

    df_temp = pd.DataFrame(dict(zip(keys, temp)), index=[0])
    return pd.concat([spreadsheet, df_temp], sort=False)

Revisar reposicion, ya que hay que acualizar el X cuando se repone, pero actualmente estamos actualizando el X en el index actual y actualizando Y en el index siguiente.

In [6]:
spreadsheet = pd.DataFrame(ss_dict)
for i in np.arange(0,30):
    spreadsheet = step(spreadsheet, i)


print('Costo pedido = ' + str(spreadsheet['CP'].sum()))
print('Costo mantenimiento = ' + str(spreadsheet['CM'].sum()))
print('Costo total = ' + str(spreadsheet['CT'].sum()))
spreadsheet


Costo pedido = 51046.89014379875
Costo mantenimiento = 9110.485559901126
Costo total = 75287.37570369989


,Tiempo,Ventas,Demora,Llegada,Pedido,Reposicion,Y,X,CP,CM,CT
0,0.0,0.0,0.0,0.000000,0.000000,0.000000,100.000000,51.300000,0.000000,513.000000,15643.000000
0,1.0,10.0,5.0,0.000000,23.000000,13.000363,103.000363,38.299637,3300.000000,382.996374,3682.996374
0,2.0,15.0,3.0,0.000000,29.599710,9.797338,97.797700,28.502300,3959.970989,285.022998,4244.993987
0,3.0,20.0,4.0,0.000000,0.000000,22.202300,100.000000,6.300000,0.000000,63.000000,63.000000
0,4.0,5.0,6.0,0.000000,14.000000,4.180639,99.180639,2.119361,2400.000000,21.193610,2421.193610
0,5.0,10.0,2.0,29.599710,33.655489,10.763725,99.944364,20.955346,4365.548877,209.553459,4575.102336
0,6.0,25.0,3.0,23.000000,30.044509,25.055636,100.000000,18.899710,4004.450878,188.997099,4193.447977
0,7.0,10.0,1.0,33.655489,23.000000,6.611946,96.611946,45.943253,3300.000000,459.432529,3759.432529
0,8.0,15.0,5.0,23.000000,0.000000,18.561795,100.173741,50.381458,0.000000,503.814581,503.814581
0,9.0,20.0,4.0,30.044509,0.000000,22.738784,102.912525,57.687183,0.000000,576.871828,576.871828
